# Initialisation de Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Exemples avec les formats de stockage") \
    .getOrCreate()
sc = spark.sparkContext

25/11/04 10:54:26 WARN Utils: Your hostname, MacBook-Air-de-Theo.local resolves to a loopback address: 127.0.0.1; using 192.0.0.2 instead (on interface en0)
25/11/04 10:54:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 10:54:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Chargement des données

## Chargement des prénoms

In [3]:
from pyspark.sql import Row

# Charge un fichier texte et convertit les lignes en "Row".
lines = sc.textFile("prenoms_sample.txt")
prenoms_as_rdd = lines.map(lambda l: l.split(";"))\
    .map(lambda p: Row(\
        sexe=int(p[0]),\
        prenom=p[1],\
        annee=int(p[2]),\
        dep=p[3],\
        effectif=int(p[4])))

# Infère le schéma et enregistre le DataFrame comme une table.
prenoms = spark.createDataFrame(prenoms_as_rdd)
prenoms.createOrReplaceTempView("prenoms")
prenoms.show()

+----+---------------+-----+---+--------+
|sexe|         prenom|annee|dep|effectif|
+----+---------------+-----+---+--------+
|   2|     MARIE-JOSÉ| 1958| 11|      14|
|   2|     MARIE-JOSÉ| 1959| 34|      24|
|   2|     MARIE-JOSÉ| 1975| 92|       4|
|   2|    MARIE-JOSEE| 1958| 57|      11|
|   2|  MARIE-JOSEPHE| 1966| 33|       3|
|   2|    MARIE-LAURE| 1968| 06|       7|
|   2|    MARIE-LAURE| 1971| 51|       9|
|   2|    MARIE-LAURE| 1974| 56|      23|
|   2|     MARIE-LISE| 1959| 56|      11|
|   2|      MARIE-LOU| 1999| 06|       3|
|   2|   MARIE-LOUISE| 1915| 68|       6|
|   2|   MARIE-LOUISE| 1917| 12|       6|
|   2|   MARIE-LOUISE| 1921| 15|      16|
|   2|   MARIE-LOUISE| 1930| 03|      10|
|   2|   MARIE-LOUISE| 1946| 73|      13|
|   2|   MARIE-LOUISE| 1955| 14|       6|
|   2|     MARIE-LUCE| 1948| 88|       3|
|   2|MARIE-MADELEINE| 1926|974|       3|
|   2|MARIE-MADELEINE| 1941| 61|       3|
|   2|   MARIE-NOËLLE| 1952| 27|       7|
+----+---------------+-----+---+--

## Chargement des départements

In [4]:
lines = sc.textFile("dpts.txt")
depts_as_rdd = lines\
    .filter(lambda l: "dep" not in l and "2A" not in l and "2B" not in l)\
    .map(lambda l: l.split(","))\
    .map(lambda p: Row(\
        dep=int(p[0]),\
        reg=int(p[1]),\
        cheflieu=p[2],\
        tncc=p[3],\
        ncc=p[4],\
        nccenr=p[6],\
        libelle=p[6]))

depts = spark.createDataFrame(depts_as_rdd)
depts.createOrReplaceTempView("depts")
depts.show()

+---+---+--------+----+--------------------+--------------------+--------------------+
|dep|reg|cheflieu|tncc|                 ncc|              nccenr|             libelle|
+---+---+--------+----+--------------------+--------------------+--------------------+
|  1| 84|   01053|   5|                 AIN|                 Ain|                 Ain|
|  2| 32|   02408|   5|               AISNE|               Aisne|               Aisne|
|  3| 84|   03190|   5|              ALLIER|              Allier|              Allier|
|  4| 93|   04070|   4|ALPES DE HAUTE PR...|Alpes-de-Haute-Pr...|Alpes-de-Haute-Pr...|
|  5| 93|   05061|   4|        HAUTES ALPES|        Hautes-Alpes|        Hautes-Alpes|
|  6| 93|   06088|   4|     ALPES MARITIMES|     Alpes-Maritimes|     Alpes-Maritimes|
|  7| 84|   07186|   5|             ARDECHE|             Ardèche|             Ardèche|
|  8| 44|   08105|   4|            ARDENNES|            Ardennes|            Ardennes|
|  9| 76|   09122|   5|              ARIEGE

# Sauvegarder les données au format parquet

## Les prénoms partitionnés par départements et années et compressés (Snappy)

In [5]:
prenoms.write\
    .partitionBy('dep', 'annee')\
        .format('parquet')\
            .save('prenomsParDeptsEtAnnees.parquet')

## Les prénoms partitionnés par départements et années et compressés (gzip)

In [6]:
prenoms.write\
    .partitionBy('dep', 'annee')\
        .option("compression", "gzip")\
            .format('parquet')\
                .save("prenomsParDeptsEtAnnees.gzip.parquet")

## Les départements

In [7]:
depts.write\
    .format("parquet")\
        .save("depts.parquet")